In [1]:
import numpy as np
import pandas as pd
import glob
from pprint import pprint
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
%matplotlib inline




pd.set_option('display.max_columns',None)

#Import Data
path = r'/homes/chh30/ECS784-PROJECT/sourcedata'
filenames = glob.glob(path+"/*.csv")

frame = pd.DataFrame

list = []

for file_ in filenames:
     df = pd.read_csv(file_,index_col=None,encoding = "ISO-8859-1",header=0)
     list.append(df)

frame = pd.concat(list,ignore_index=True,sort=False)

frame.head()
print(frame.shape)

(10216388, 15)


In [2]:
# Clean Data


##Drop records with null value
frame.dropna(axis=0, subset=["Rental Id",
                             "Bike Id",
                             "Start Date",
                             "StartStation Id",
                             "StartStation Name",
                             "End Date",
                             "EndStation Id",
                             "EndStation Name",
                             "Duration"], inplace=True)


## Extra drop for duplicates
frame.drop_duplicates(inplace=True)



## Formatting column
frame["Bike Id"] = frame["Bike Id"].astype(int)
frame["Rental Id"] = frame["Rental Id"].astype(int)
frame["EndStation Id"] = frame["EndStation Id"].astype(int)
frame["StartStation Id"] = frame["StartStation Id"].astype(int)
frame["EndStation Id"] = frame["EndStation Id"].astype(int)
frame["Duration"] = frame["Duration"].astype(int)

## Clean up data which StartStation Id = EndStation Id
frame = frame[frame["StartStation Id"] != frame["EndStation Id"]]


## Grab the column we need
frame = frame.loc[:,('Start Date',
                           'StartStation Id',
                           'StartStation Name',
                           'End Date',
                           'EndStation Id',
                           'EndStation Name',
                           'Duration')]


## Change 'Start Date', 'End Date' Data Type as datetime
frame['Start Date'] = pd.to_datetime(frame['Start Date'],format='%d/%m/%Y %H:%M',dayfirst=True)
frame['End Date'] = pd.to_datetime(frame['End Date'],format='%d/%m/%Y %H:%M',dayfirst=True)


## Grab data between 2016/01/01 00:00:00 to 2016/12/31 23:59:59
begin = pd.Timestamp('2016-01-01 00:00:00')
end = pd.Timestamp('2016-12-31 23:59:59')
mask=(frame['Start Date'] >= begin)&(frame['Start Date'] <= end)
frame.loc[mask]


frame.describe()

,StartStation Id,EndStation Id,Duration
count,9.533099e+06,9.533099e+06,9.533099e+06
mean,3.505212e+02,3.474602e+02,1.187304e+03
std,2.309940e+02,2.307544e+02,6.620339e+03
min,1.000000e+00,1.000000e+00,-3.540000e+03
25%,1.540000e+02,1.540000e+02,4.800000e+02
50%,3.160000e+02,3.100000e+02,7.800000e+02
75%,5.410000e+02,5.400000e+02,1.260000e+03
max,8.200000e+02,8.200000e+02,2.674020e+06


In [3]:
# Missing data checking

missing_data = frame.isnull()

for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print("")

Start Date
False    9533099
Name: Start Date, dtype: int64

StartStation Id
False    9533099
Name: StartStation Id, dtype: int64

StartStation Name
False    9533099
Name: StartStation Name, dtype: int64

End Date
False    9533099
Name: End Date, dtype: int64

EndStation Id
False    9533099
Name: EndStation Id, dtype: int64

EndStation Name
False    9533099
Name: EndStation Name, dtype: int64

Duration
False    9533099
Name: Duration, dtype: int64



In [4]:
# See which journey between two station has the most transaction record
journeyCnt = frame.groupby(['StartStation Name','StartStation Id','EndStation Name','EndStation Id']).size().reset_index(name='Rental_Counts').nlargest(5,columns='Rental_Counts').tail(5)
journeyCnt

,StartStation Name,StartStation Id,EndStation Name,EndStation Id,Rental_Counts
37020,"Black Lion Gate, Kensington Gardens",307,"Palace Gate, Kensington Gardens",404,4497
175072,"Hyde Park Corner, Hyde Park",191,"Triangle Car Park, Hyde Park",248,4223
36902,"Black Lion Gate, Kensington Gardens",307,"Hyde Park Corner, Hyde Park",191,4192
174481,"Hyde Park Corner, Hyde Park",191,"Albert Gate, Hyde Park",303,4006
243230,"Palace Gate, Kensington Gardens",404,"Black Lion Gate, Kensington Gardens",307,3911


In [5]:
# Get journey data between Black Lion Gate and Palace Gate
journeyData = frame[(frame['StartStation Id']==307)&(frame['EndStation Id']==404)]


# Spilt Year,Month, Hour from Start Date
journeyData['Year']= journeyData['Start Date'].map(lambda x:x.year)
journeyData['Month'] = journeyData['Start Date'].map(lambda x: x.month)
journeyData['Hour'] = journeyData['Start Date'].map(lambda x: x.hour)

# Cleaning the transaction between 7am to 21pm
hourmask=(journeyData['Hour']>=7)&(journeyData['Hour']<=21)
journeyData.loc[hourmask]

# Grab needed column
df1 = journeyData.loc[:,(  'Start Date',
                           'Year',
                           'Month',
                           'Hour',
                           'Duration')]
df1.tail(10)


/import/linux/python/3.6.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/import/linux/python/3.6.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/import/linux/python/3.6.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.p

,Start Date,Year,Month,Hour,Duration
10197702,2016-05-17 14:25:00,2016,5,14,420
10197703,2016-05-17 14:25:00,2016,5,14,420
10198536,2016-05-17 15:02:00,2016,5,15,300
10199050,2016-05-17 15:22:00,2016,5,15,2880
10199749,2016-05-17 15:47:00,2016,5,15,1200
10199750,2016-05-17 15:47:00,2016,5,15,1200
10207037,2016-05-17 18:06:00,2016,5,18,240
10209777,2016-05-17 18:44:00,2016,5,18,240
10212557,2016-05-17 19:45:00,2016,5,19,240
10212584,2016-05-17 19:46:00,2016,5,19,1620


In [7]:
dfRentalCnt = df1.groupby(['Year','Month','Hour']).size().reset_index(name='HourlyCount')
dfRentalCnt['HourlyCount'] = dfRentalCnt['HourlyCount']/100

for i in range(12):
    x = i+1
    dfCSV = dfRentalCnt.loc[dfRentalCnt['Month']==x,('Hour','HourlyCount')]
    Path = r'/homes/chh30/ECS784-PROJECT/Export_Data/WeekdayRental'+str(x)+'.csv'
    dfCSV.to_csv(Path,index=False)
    
dfRentalCnt.head(12)

,Year,Month,Hour,HourlyCount
0,2016,1,7,0.12
1,2016,1,8,0.19
2,2016,1,9,0.07
3,2016,1,10,0.12
4,2016,1,11,0.15
5,2016,1,12,0.26
6,2016,1,13,0.30
7,2016,1,14,0.22
8,2016,1,15,0.17
9,2016,1,16,0.18


In [10]:
import calendar as cl
dfDuration = df1
dfDuration['monthrange'] = (dfDuration['Start Date'].map(lambda x: cl.monthrange(x.year,x.month)[1]))
dfDuration['Duration'] = dfDuration['Duration']/dfDuration['monthrange']
dfDuration = dfDuration.groupby(['Year','Month','Hour'])['Duration'].agg(['sum'])
dfDuration.tail(10)
pathDuration = r'/homes/chh30/ECS784-PROJECT/Export_Data/Duration.csv'
dfDuration.to_csv(pathDuration,index=False)

dfDuration.tail(10)

sum
Year Month Hour          
2016 12    18    0.016112
           19    0.008056
2017 1     8     0.054379
           10    0.294049
           11    0.096673
           12    0.008056
           13    0.054379
           14    0.116814
           15    0.217515
           16    0.034239